In [ ]:
from azureml.core import Workspace, Experiment, Dataset, Model
from azureml.core import Environment

In [ ]:
ws = Workspace.from_config()

In [ ]:
experiment_name = 'livedoor-news-classification-BERT-pipeline-inference'
experiment = Experiment(ws, experiment_name)

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Compute Cluster の名称
amlcompute_cluster_name = "gpucluster"

# クラスターの存在確認
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    
except ComputeTargetException:
    print('指定された名称のクラスターが見つからないので新規に作成します.')
    compute_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_NC6_V3",
                                                           max_nodes = 4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

In [ ]:
my_dataset = Dataset.get_by_name(ws, "livedoor").keep_columns(["text"])


In [ ]:
project_folder = '../code/deployment'

In [ ]:
envs = Environment.list(workspace=ws)
[env if env.startswith("AzureML") else None for env in envs]

In [ ]:
env =  Environment.get(workspace=ws, name='AzureML-AutoML-DNN-GPU')

In [ ]:
from azureml.core.runconfig import RunConfiguration
run_config = RunConfiguration()
run_config.environment.docker.enabled = True
run_config.environment = env

In [ ]:
from azureml.pipeline.steps import PythonScriptStep

ds_input = my_dataset.as_named_input('input1')

inferenceStep = PythonScriptStep(
    script_name="inference_script.py",
    inputs=[ds_input],
    compute_target=compute_target,
    source_directory=project_folder,
    runconfig=run_config,
    allow_reuse=True
)

In [ ]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[inferenceStep])

In [ ]:
pipeline.validate()
print("Pipeline validation complete")

In [ ]:
pipeline_run = experiment.submit(pipeline)

In [ ]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()